# Auto-Evaluation Example

To view this example on Google Colab, see [here](https://colab.research.google.com/github/hegelai/prompttools/blob/main/examples/notebooks/AutoEval.ipynb).

## Installations

In [ ]:
# !pip install --quiet --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use a real OpenAI key, so for now we'll set them to empty strings.

In [ ]:
import os

os.environ["DEBUG"] = "1"  # Set to "1" if you want to use debug mode.
os.environ["OPENAI_API_KEY"] = ""

Then we'll import the relevant `prompttools` modules to set up our experiment.

In [ ]:
from typing import Dict
from prompttools.harness import PromptTemplateExperimentationHarness
from prompttools.experiment import OpenAICompletionExperiment
from prompttools.selector.prompt_selector import PromptSelector

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [ ]:
instructions = [
    """
Answer the following question.
If it is not prime, give its decomposition.
""",
    """
Answer the following question.
""",
]

inputs = [{"input": "is 17077 a prime number"}, {"input": "Is 17077 prime?"}]

selectors = [PromptSelector(instructions[i], inputs[j]) for i in range(len(instructions)) for j in range(len(inputs))]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [ ]:
experiment = OpenAICompletionExperiment(["text-davinci-003"], selectors)

We can then run the experiment to get results.

In [ ]:
experiment.run()
experiment.visualize()

,prompt,response(s),latency
0,"\nAnswer the following question. \nIf it is not prime, give its decomposition.","\n\n93\nIt is not a prime number, its decomposition is 3",1.993576
1,"\nAnswer the following question. \nIf it is not prime, give its decomposition.",\n\n45\n\n45 = 3 x 3 x 5,0.908900
2,\nAnswer the following question.,"\n\nWhat is the meaning of the phrase ""catching some z's""?\n",0.922788
3,\nAnswer the following question.,\n\nIn what ways might information literacy skills help students succeed in their studies?,1.075018


## Auto-Evaluate the model response

To evaluate the model response, we can define an eval method that passes the input and response into another LLM to get feedback.

In [ ]:
from prompttools.utils import autoeval_binary_scoring

Finally, we can evaluate and visualize the results.

In [ ]:
experiment.evaluate("followed_directions", autoeval_binary_scoring, {"prompt_column_name": "prompt"})
experiment.visualize()

,prompt,response(s),latency,followed_directions
0,"\nAnswer the following question. \nIf it is not prime, give its decomposition.","\n\n93\nIt is not a prime number, its decomposition is 3",1.993576,0.0
1,"\nAnswer the following question. \nIf it is not prime, give its decomposition.",\n\n45\n\n45 = 3 x 3 x 5,0.908900,1.0
2,\nAnswer the following question.,"\n\nWhat is the meaning of the phrase ""catching some z's""?\n",0.922788,0.0
3,\nAnswer the following question.,\n\nIn what ways might information literacy skills help students succeed in their studies?,1.075018,0.0
